In [15]:
import nbformat as nbf
import lxml.etree as etree
import sys
import requests
import os
import string
import hashlib
import urllib2
import ast


nb_source = 'input-Copy1.ipynb'
nb = nbf.read(nb_source, 4)

In [11]:
def ciopgetparam(name):
    
    return 'KR'

In [4]:
nb['cells'][2]

{u'cell_type': u'code',
 u'execution_count': 4,
 u'metadata': {u'collapsed': False},
 u'outputs': [],
 u'source': u"pa_code = dict([('id', 'pa_code'),\n               ('value', 'DO'),\n               ('title', 'Protected Area code'),\n               ('abstract', 'Protected Area code (one of DO,...)')])"}

### Loop all cells and retain the ones that assign a dict

In [7]:
nb['cells'][5]['source']

u"data_path = '/workspace/data'"

In [6]:
import ast

In [8]:
root = ast.parse(str(nb['cells'][5]['source']))
names = sorted({node.id for node in ast.walk(root) if isinstance(node, ast.Name)})

In [11]:
names[0]

'data_path'

In [10]:
exec(str(nb['cells'][2]['source']))

In [11]:
eval(names[1]).keys()

['abstract', 'id', 'value', 'title']

### execute the cell to create the object

In [4]:
exec(str(nb['cells'][3]['source']))

In [99]:
pa_code  = None

In [40]:
if 'title' in pa_code.keys() and 'abstract' in pa_code.keys() and 'id' in pa_code.keys():
    print 'it is a parameter'
    
    pa_code['value'] = ciopgetparam(pa_code['id'])
    
    a = 'dict(['

    for i, keys in enumerate(pa_code):
        if i == 0: 
            a = a + '( "%s", "%s")' % (keys, pa_code[keys]) 
        else:
            a = a + ',( "%s", "%s")' % (keys, pa_code[keys]) 
        
    a = a + '])'
    
    nb['cells'][2]['source'] = '%s = %s' % (names[0], a)
    

it is a parameter


In [41]:
nb['cells'][2]['source']

'dict = dict([( "abstract", "Protected Area code (one of DO,...)"),( "id", "pa_code"),( "value", "KR"),( "title", "Protected Area code")])'

In [93]:
exec(str(nb['cells'][2]['source']))

TypeError: 'dict' object is not callable

In [43]:
pa_code

{'abstract': 'Protected Area code (one of DO,...)',
 'id': 'pa_code',
 'title': 'Protected Area code',
 'value': 'KR'}

### Application.xml

In [44]:
import lxml.etree as etree
import sys
import requests
import os
import string
import hashlib
import urllib2

In [47]:
template = 'application_template.xml'
tree = etree.parse(template)
root = tree.getroot()

In [48]:
xml_string = '<parameter id="%s" title="%s" abstract="%s" maxOccurs="1" scope="runtime" type="LiteralData">%s</parameter>' % (
        pa_code['id'], pa_code['title'], pa_code['abstract'],pa_code['value'] )

In [49]:
xml_string

'<parameter id="pa_code" title="Protected Area code" abstract="Protected Area code (one of DO,...)" maxOccurs="1" scope="runtime" type="LiteralData">KR</parameter>'

In [58]:
el_default_params = root.xpath('/application/jobTemplates/jobTemplate/defaultParameters', 
                             namespaces={})
        

In [59]:
el_default_params

[<Element defaultParameters at 0x7f4cab9f7680>]

In [61]:
el_default_params[0].addnext(etree.fromstring(xml_string))

In [63]:
etree.tostring(tree, pretty_print=True)

'<application xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" id="application">\n  <jobTemplates>\n    <jobTemplate id="notebook">\n      <streamingExecutable>/application/notebook/run</streamingExecutable>\n      <defaultParameters>\n     </defaultParameters><parameter id="pa_code" title="Protected Area code" abstract="Protected Area code (one of DO,...)" maxOccurs="1" scope="runtime" type="LiteralData">KR</parameter>\n     <defaultJobconf>\n        <property id="mapred.task.timeout">9900000000000</property>\n      </defaultJobconf>\n    </jobTemplate>\n  </jobTemplates>\n  <workflow id="dcs-notebook" title="Notebook driven application" abstract="Notebook driven application">\n    <workflowVersion>1.0</workflowVersion>\n    <node id="notebook">\n      <job id="notebook"/>\n      <sources>\n        <source id="source" title="Input references" abstract="Input references" refid="string:list" scope="runtime">https://catalog.terradue.com/s

### Set workflow information

In [83]:
root = ast.parse(str(nb['cells'][3]['source']))
names = sorted({node.id for node in ast.walk(root) if isinstance(node, ast.Name)})

In [86]:
names[1]

'service'

In [16]:
names[1]

'pa_code'

In [23]:
eval('pa_code')['id']

'pa_code'

### loop all cells

In [14]:
import nbformat as nbf
import lxml.etree as etree
import sys
import requests
import os
import string
import hashlib
import urllib2
import ast

def ciopgetparam(name):
    
    return 'KR'

nb_source = 'input-Copy1.ipynb'
nb = nbf.read(nb_source, 4)

data_path = '/tmp'
input_identifier = 'some_identifier'
input_reference = 'some_reference'

action = 'runtime'
#action = 'descriptor'

if action == 'descriptor':
    template = 'application_template.xml'
    tree = etree.parse(template)
    root = tree.getroot()

for index, cell in enumerate(nb['cells']):
    print index
    
    if str(cell['cell_type']) == 'code': 
        
        try:
            root_ast = ast.parse(str(cell['source']))
            names = sorted({node.id for node in ast.walk(root_ast) if isinstance(node, ast.Name)})
          
            if names[0] == 'data_path' and action == 'runtime':
                print('update datapath')
                cell['source'] = 'data_path = %s' % data_path  
        
            if names[0] == 'input_identifier' and action == 'runtime':
                print('update input_identifier')
                cell['source'] = 'input_identifier = %s' % input_identifier  
                
            if names[0] == 'input_reference' and action == 'runtime':
                print('update input_reference')   
                cell['source'] = 'input_reference = %s' % input_reference  
                
            if names[0] == 'dict':
                print(str(cell['source']))   
                exec(str(cell['source']))
                
                if 'title' in eval(names[1]).keys() and 'abstract' in eval(names[1]).keys() and 'id' in eval(names[1]).keys():
                    
                    if 'value' in eval(names[1]).keys():
                        # it's a parameter
                        if action == 'descriptor':
            
                            xml_string = '<parameter id="%s" title="%s" abstract="%s" maxOccurs="1" scope="runtime" type="LiteralData">%s</parameter>' % (eval(names[1])['id'], 
                                  eval(names[1])['title'], 
                                  eval(names[1])['abstract'], 
                                  eval(names[1])['value'])
                        
                            el_default_params = root.xpath('/application/jobTemplates/jobTemplate/defaultParameters', 
                                 namespaces={})
                        
                            el_default_params[0].append(etree.fromstring(xml_string))
                        
                    
                        if action == 'runtime':
                        
                            eval(names[1])['value'] = ciopgetparam(eval(names[1])['id'])
    
                            new_source = 'dict(['

                            for i, keys in enumerate(eval(names[1])):
                                if i == 0: 
                                    new_source = new_source + '( "%s", "%s")' % (keys, eval(names[1])[keys]) 
                                else:
                                    new_source = new_source + ',( "%s", "%s")' % (keys, eval(names[1])[keys]) 
        
                            new_source = new_source + '])'
    
                            cell['source'] = '%s = %s' % (names[1], new_source)
                        
                    else:
                        if action == 'descriptor':
                            # it's the service definition
                            el_workflow = root.xpath('/application/workflow',
                                                namespaces={}) 
         
                            el_workflow[0].attrib['id'] = eval(names[1])['id']
                            el_workflow[0].attrib['title'] = eval(names[1])['title']
                            el_workflow[0].attrib['abstract'] = eval(names[1])['abstract']
                    
        except SyntaxError:
            continue
            
        if index == 2: 
            print cell['source']
    
etree.tostring(tree, pretty_print=True)    

0
1
2
pa_code = dict([('id', 'pa_code'),
               ('value', 'DO'),
               ('title', 'Protected Area code'),
               ('abstract', 'Protected Area code (one of DO,...)')])
pa_code = dict([( "abstract", "Protected Area code (one of DO,...)"),( "id", "pa_code"),( "value", "KR"),( "title", "Protected Area code")])
3
service = dict([('title', 'NDVI BOA '),
                ('abstract', 'Sentinel-2 NDVI BOA'),
                ('id', 'ndvi_boa')])
4
pa_name = dict([('id', 'pa_name'),
               ('value', 'Donana'),
               ('title', 'Protected Area name'),
               ('abstract', 'Protected Area name (one of Donana,...)')])
5
update datapath
6
update input_identifier
7
update input_reference
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


NameError: name 'tree' is not defined

In [3]:
root

### Scratchpad

In [29]:
a = 'dict(['

for i, keys in enumerate(pa_code):
    if i == 0: 
        a = a + '( %s, "%s")' % (keys, pa_code[keys]) 
    else:
        a = a + ',( %s, "%s")' % (keys, pa_code[keys]) 
        
a = a + '])'

In [30]:
a

'dict([( abstract, "Protected Area code (one of DO,...)"),( id, "pa_code"),( value, "KR"),( title, "Protected Area code")])'

In [79]:
nb['cells'][2]

{u'cell_type': u'code',
 u'execution_count': 2,
 u'metadata': {u'collapsed': False},
 u'outputs': [],
 u'source': u"pa_code = dict([('id', 'pa_code'),\n               ('value', 'DO'),\n               ('title', 'Protected Area code'),\n               ('abstract', 'Protected Area code (one of DO,...)')])"}

In [4]:
nb['cells'][3]

{u'cell_type': u'code',
 u'execution_count': 2,
 u'metadata': {u'collapsed': True},
 u'outputs': [],
 u'source': u'nc = 5'}

In [5]:
nb['cells'][5]

{u'cell_type': u'code',
 u'execution_count': 3,
 u'metadata': {u'collapsed': True},
 u'outputs': [],
 u'source': u'maxit = 1000'}

In [6]:
nb['cells'][7]

{u'cell_type': u'code',
 u'execution_count': 5,
 u'metadata': {u'collapsed': True},
 u'outputs': [],
 u'source': u"data_path = '/tmp/workspace/data'"}

In [7]:
nb['cells'][8]

{u'cell_type': u'code',
 u'execution_count': 6,
 u'metadata': {u'collapsed': True},
 u'outputs': [],
 u'source': u"s2_identifier = 'S2A_MSIL1C_20170909T060631_N0205_R134_T42SVG_20170909T061506'"}

In [8]:
nb['cells'][9]

{u'cell_type': u'code',
 u'execution_count': None,
 u'metadata': {u'collapsed': True},
 u'outputs': [],
 u'source': u"s2_reference = ''"}